In [143]:
# use geopandas to designate an area of interest around a given coordinate point instead of cropping by sight
#https://towardsdatascience.com/satellite-imagery-access-and-analysis-in-python-jupyter-notebooks-387971ece84b

#Libraries used:
#MatplotLib: 
# python -m pip install -U matplotlib
# https://matplotlib.org/
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
#makes the images display inside this notebook:
%matplotlib inline 
import matplotlib.ticker as mtick
from matplotlib.ticker import PercentFormatter
import matplotlib.collections as collections
import matplotlib.patches as patches
from PIL import Image

#tkinter to support gui dialogue boxes, etc
import tkinter as tk
from tkinter import filedialog
from tkinter import ttk
from tkinter.simpledialog import askinteger

#Numpy
import numpy as np

#shapely (interface to GEOS) "pip install shapely"
from shapely.geometry import Point
from shapely import geometry
from shapely.geometry import Polygon

#Fiona
import fiona

#GeoPandas
import geopandas as gpd
# pip install geopandas
import folium 
# pip install folium


In [144]:
#case study
#**Name:** 			        Ackerpulco
#**Link:**			        http://ackerpulco-farm.de/
#**Location:**			    Schlossstraße 5, 15518, Alt-Madlitz, Germany
#**Coordinates:**		    52.37962462071622, 14.280646369492525 --- 52°22'46.7"N 14°16'50.3"E

lat = 52.37962462071622
lon = 14.280646369492525
center = Point((lat, lon))
#GeoSeries = "52.37962462071622, 14.280646369492525"
#load image
#redImageOriginal = mpimg.imread("""C:/Users/clair/Climate Farmers/Imagery/StudyArea1/HLS.S30.T33UVU.2021081T101649.v1.5.B04.tif""")

In [145]:
# create a basic map of the area

tooltip = "Click me!"
m = folium.Map([lat, lon], zoom_start=16)
folium.Marker(
    [lat, lon], popup="<b>Ackerpulco</b>", tooltip=tooltip
).add_to(m)
m



In [146]:
#designate an area around our given coordinate point (HELP)

m = folium.Map([lat, lon], zoom_start=16)
folium.Marker([lat, lon], popup="<b>Ackerpulco</b>", tooltip=tooltip).add_to(m)

##draw a circle
##areaOfInterest=folium.Circle(location=[52.37962462071622, 14.280646369492525], popup='Ackerpulco', fill_color='#000', radius=500, weight=2, color="#000").add_to(m)

##draw a rectangle centered around lat, lon (the distance a degree of longitude is depends on the latitude, we can update this later)
degree=0.005
latBounds = (lat-(degree/2), lat+(degree/2), lat+(degree/2), lat-(degree/2))
lonBounds = (lon-(degree/2), lon-(degree/2), lon+(degree/2), lon+(degree/2))

#try as a polgyon:
bounds = zip(latBounds,lonBounds)
#print(tuple(bounds))
#folium.Polygon(bounds).add_to(m)

#try with GeoDataFrame
polygonBounds=Polygon(bounds)
crs = {'init': 'epsg:32721'}
polygon = gpd.GeoDataFrame(index=[0], crs=crs, geometry=[polygonBounds])   
type(polygon)
folium.GeoJson(polygon).add_to(m)
m


C:\Users\clair\AppData\Local\Programs\Python\Python38-32\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [45]:
#add satellite data

#from https://towardsdatascience.com/satellite-imagery-access-and-analysis-in-python-jupyter-notebooks-387971ece84b:
crs = {'init': 'epsg:32721'}
areaOfInterest_proj = areaOfInterest.to_crs({'init': 'epsg:32633'})


AttributeError: 'Polygon' object has no attribute 'to_crs'

In [55]:
#crop or extract area

#from https://towardsdatascience.com/satellite-imagery-access-and-analysis-in-python-jupyter-notebooks-387971ece84b:
# 


with rio.open("RGB.tiff") as src:
    out_image, out_transform = rio.mask.mask(src, areaOfInterest_proj.geometry,crop=True)
    out_meta = src.meta.copy()
    out_meta.update({"driver": "GTiff",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform})
    
with rasterio.open("RGB_masked.tif", "w", **out_meta) as dest:
    dest.write(out_image)

NameError: name 'rio' is not defined